# Checking the values of Wigner D and G matrices

### Definition of Wigner D
I define the Wigner D matrix as $D^{(\ell)}_{m',m}(R) = \langle \ell m'| R |\ell m\rangle$, with $R$ an active rotation acting on the function $|\ell m\rangle$, a complex spherical harmonic: $$ R \cdot |\ell m\rangle = \sum_{m'} D_{m' m}^{(\ell)}(R) |\ell m'\rangle,$$ where $R \cdot Y_{\ell}^{m}(\hat x) = Y_\ell^m(R^{-1} \hat x)$.

The test function vsdm.testD_lm calculates $Y_\ell^m(R^{-1} \hat x)$ directly, and compares it to the sum $\sum_{m'} M_{m' m}^{(\ell)}(R) |\ell m'\rangle$, for matrices $M_{ij} = D_{ij}$, $M_{ij} = D_{ij}^\star$, $M_{ij} = D_{ji}$, $M_{ij} = D_{ji}^\star$, to see which (if any) of these results match. In the current version (v1.0.14) of 'spherical', the object returned by Wigner.D(l, m, mp) is the complex conjugate of D(l, m, mp), $M_{ij} = D_{ij}^\star$. To adjust for this, the calculation of $G^{(\ell)}_{m, m'}$ undoes the complex conjugation before assembling $G^{(\ell)}$. The test function vsdm.testG_lm verifies whether this has been done correctly, by calculating $$ R \cdot |\ell m\rangle = \sum_{m'} G_{m' m}^{(\ell)}(R) |\ell m'\rangle,$$ for real spherical harmonics $|\ell m \rangle = Y_{\ell m}$. 

Rotations are implemented using quaternions. For a rotation of $\eta$ about the axis $\hat n$, the quaternion $$R = \left( \cos(\eta/2) ,~ \sin(\eta/2) \cdot \hat n \right)$$ rotates imaginary vectors $\vec v = x i + yj + zk$ to their image $R \vec v$ as: $$R \vec v = R \vec v R^{-1}.$$

In [1]:
import numpy as np
from spherical import Wigner 
import quaternionic
import sys
import time 
# sys.path.insert(0,'../') #load the local version of vsdm
import vsdm 
import numba 

vsdm.__version__

'0.3.5'

In [2]:
vsdm.testG_lm(13, -3, printout=True)

Ylm(R^(-1) * x): -0.46737809071889735
(M) G_(k,m)*Ylk(x): -0.467378090718897
(M) difference: -3.3306690738754696e-16
(V) G_(k,m)*Ylk(x): -0.467378090718897
(V) difference: -3.3306690738754696e-16


(-3.3306690738754696e-16, -3.3306690738754696e-16)

In [3]:
vsdm.testD_lm(60, -1, printout=True)

Ylm(R^(-1) * x): (0.044294389411309754+0.27431158162808184j)
D_(k,m)*Ylk(x): (-0.14762269043440543+0.054155799814597214j)
D_(m,k)*Ylk(x): (-0.3209041050209531+0.17091069670545433j)
D*_(k,m)*Ylk(x): (0.044294389411310364+0.27431158162807856j)
D*_(m,k)*Ylk(x): (-0.12411257112208327+0.20899457712434355j)
differences:
	D: (0.19191707984571518+0.2201557818134846j)
	D_T: (0.3651984944322629+0.1034008849226275j)
	D_star: (-6.106226635438361e-16+3.2751579226442118e-15j)
	D_dagger: (0.16840696053339302+0.06531700450373829j)
version of Wigner D(R) provided by spherical.D(R): ['D_star']


['D_star']

In [4]:
def mxG_l(gvec, l, lmod=1):
    ix_start = vsdm.Gindex(l, -l, -l, lmod=lmod)
    ix_end = vsdm.Gindex(l, l, l, lmod=lmod)
    return gvec[ix_start:ix_end+1].reshape((2*l+1, 2*l+1))


In [5]:
wigG = vsdm.WignerG(7)
R = quaternionic.array([2, 5, 3, 7]).normalized
wG = wigG.G(R)
mxG_l(wG, 1)

array([[-0.70114943,  0.25287356,  0.66666667],
       [ 0.71264368,  0.2183908 ,  0.66666667],
       [ 0.02298851,  0.94252874, -0.33333333]])

## Checking Speed

In [6]:
@numba.jit(nopython=True)
def dot_GR_K(gR_lmk, k_lmk, out):
    n_r = len(gR_lmk)
    n_i = len(k_lmk)
    for r in range(n_r):
        for i in range(n_i):
            out[r] += gR_lmk[r, i] * k_lmk[i]

def testSpeed(N):
    n_segs = 30
    seg_len = 200
    mxG = np.random.rand(N, n_segs*seg_len)
    mxH = np.random.rand(n_segs*seg_len)
    start_end_ix = [[j, j+seg_len] for j in range(0, n_segs*seg_len, seg_len)]
    # current version: 
    t0 = time.time()
    mu_l = []
    for vecG in mxG:
        mu_l_R = []
        for ix in range(n_segs):
            start, end = start_end_ix[ix]
            g_l = vecG[start:end]
            h_l = mxH[start:end]
            mu_l_R += [g_l @ h_l]
        mu_l += [mu_l_R]
    tCurrent = time.time() - t0 
    print('array slicing version:', tCurrent)
    # alternative slicing: 
    t0 = time.time()
    mu_l = np.zeros((N, n_segs))
    for l in range(len(start_end_ix)):
        start, end = start_end_ix[l]
        g_l = mxG[:, start:end]
        h_l = mxH[start:end]
        mu_l[:, l] = g_l @ h_l
    tAltSlice = time.time() - t0
    print('alt slicing version:', tAltSlice)
    # alternative:
    t0 = time.time()
    dH = {}
    dG = {}
    for l in range(n_segs):
        start, end = start_end_ix[l]
        dH[l] = mxH[start:end]
        dG[l] = mxG[:, start:end]
    tPrep = time.time() - t0 
    print('prep time for dict:', tPrep)
    # dot product within dict:
    t0 = time.time()
    mu_l = np.zeros((N, n_segs))
    for l in range(n_segs):
        mu_l[:, l] = dG[l] @ dH[l]
    tDict = time.time() - t0
    print('dictionary version:', tDict)
    # jit version 
    t0 = time.time()
    mu_l = np.zeros((N, n_segs))
    for l in range(n_segs):
        mu_Rl = np.zeros(N)
        dot_GR_K(dG[l], dH[l], mu_Rl)
        mu_l[:,l] = mu_Rl
    tJIT = time.time() - t0
    print('dict + jit version:', tJIT)
    return tCurrent, tJIT
    


In [7]:
vG = np.random.rand(14, 12)
vK = np.random.rand(12)
out = np.zeros(14)
dot_GR_K(vG, vK, out)

In [8]:
testSpeed(10000)

array slicing version: 0.5322411060333252
alt slicing version: 0.027164220809936523
prep time for dict: 3.2901763916015625e-05
dictionary version: 0.02332282066345215
dict + jit version: 0.18120312690734863


(0.5322411060333252, 0.18120312690734863)

In [9]:
def speedRunG(N, ellMax, lmod=2):
    wG = vsdm.WignerG(ellMax, lmod=lmod)
    t0 = time.time()
    for i in range(N):
        R = quaternionic.array(np.random.rand(4)).normalized
        wG.G(R, save=True)
    tG = time.time() - t0 
    print('tWignerG[{}]:'.format(ellMax), tG)
    wigG = np.array(wG.G_array)
    fakeK = wigG[0] * np.pi
    # run through list
    t0 = time.time()
    mu = np.zeros(N)
    for r in range(N):
        mu[r] = wigG[r] @ fakeK 
    tR = time.time() - t0
    print('tRotations:', tR)
    return tG/N , tR/N 

#

In [10]:
timesGR = {}
for l in [12, 16, 24, 36]:
    for lmod in [1, 2]:
        timesGR[(l, lmod)] = speedRunG(1000, l, lmod=lmod)

tWignerG[12]: 2.6440680027008057
tRotations: 0.003094911575317383
tWignerG[12]: 1.4985098838806152
tRotations: 0.0020771026611328125
tWignerG[16]: 4.90713095664978
tRotations: 0.004519939422607422
tWignerG[16]: 2.71913480758667
tRotations: 0.003072023391723633
tWignerG[24]: 13.22105598449707
tRotations: 0.051187992095947266
tWignerG[24]: 7.057474851608276
tRotations: 0.04914093017578125
tWignerG[36]: 38.71676802635193
tRotations: 0.27673983573913574
tWignerG[36]: 20.58334732055664
tRotations: 0.043900251388549805


In [11]:
for (l, lmod) in timesGR.keys():
    tG, tR = timesGR[(l, lmod)]
    print('{}:   tG: {:.3f} ms,\ttR: {:.3f} us'.format((l, lmod), tG*1e3, tR*1e6))

(12, 1):   tG: 2.644 ms,	tR: 3.095 us
(12, 2):   tG: 1.499 ms,	tR: 2.077 us
(16, 1):   tG: 4.907 ms,	tR: 4.520 us
(16, 2):   tG: 2.719 ms,	tR: 3.072 us
(24, 1):   tG: 13.221 ms,	tR: 51.188 us
(24, 2):   tG: 7.057 ms,	tR: 49.141 us
(36, 1):   tG: 38.717 ms,	tR: 276.740 us
(36, 2):   tG: 20.583 ms,	tR: 43.900 us


Note that the size of the WignerG object increases as $\ell_{max}^3$:

In [12]:
[vsdm.Gindex(l, l, l, lmod=1)+1 for l in range(0, 61, 10)]

[1, 1771, 12341, 39711, 91881, 176851, 302621]